In [1]:
import pandas as pd
import os

In [2]:
raw_data_path = '../AFOLU/yf_agrc_raw_data'
raw_data_file_name = 'FAOSTAT_crop_data.csv'

In [3]:
# Import raw data from FAO csv file
raw_data_df = pd.read_csv(os.path.join(raw_data_path, raw_data_file_name))
raw_data_df.head()

,Domain Code,Domain,Area Code (ISO3),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,AFG,Afghanistan,5412,Yield,1371.0,"Almonds, in shell",2011,2011,kg/ha,4500.0,A,Official figure
1,QCL,Crops and livestock products,AFG,Afghanistan,5412,Yield,1371.0,"Almonds, in shell",2012,2012,kg/ha,4596.0,A,Official figure
2,QCL,Crops and livestock products,AFG,Afghanistan,5412,Yield,1371.0,"Almonds, in shell",2013,2013,kg/ha,2991.0,A,Official figure
3,QCL,Crops and livestock products,AFG,Afghanistan,5412,Yield,1371.0,"Almonds, in shell",2014,2014,kg/ha,1999.6,A,Official figure
4,QCL,Crops and livestock products,AFG,Afghanistan,5412,Yield,1371.0,"Almonds, in shell",2015,2015,kg/ha,1652.1,A,Official figure


In [4]:
# Drop irrelevant columns and modify names to match SISEPUEDE input style
yf_df = raw_data_df[['Area Code (ISO3)', 'Area', 'Item', 'Year', 'Value']]
yf_df = yf_df.rename(columns={'Area Code (ISO3)':'iso_code3', 'Area':'Nation'})
yf_df.head()

,iso_code3,Nation,Item,Year,Value
0,AFG,Afghanistan,"Almonds, in shell",2011,4500.0
1,AFG,Afghanistan,"Almonds, in shell",2012,4596.0
2,AFG,Afghanistan,"Almonds, in shell",2013,2991.0
3,AFG,Afghanistan,"Almonds, in shell",2014,1999.6
4,AFG,Afghanistan,"Almonds, in shell",2015,1652.1


In [5]:
yf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107993 entries, 0 to 107992
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   iso_code3  107993 non-null  object 
 1   Nation     107993 non-null  object 
 2   Item       107993 non-null  object 
 3   Year       107993 non-null  int64  
 4   Value      107993 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.1+ MB


In [6]:
# Checking for null values
yf_df.isnull().sum()

iso_code3    0
Nation       0
Item         0
Year         0
Value        0
dtype: int64

In [7]:
# Importing crosswalk file
cw_df = pd.read_csv("https://raw.githubusercontent.com/jcsyme/sisepuede/main/ref/data_crosswalks/fao_crop_categories.csv")
cw_df = cw_df.rename(columns = {"fao_crop" : "Item", "``$CAT-AGRICULTURE$``" : "sisepuede_item"})
cw_df["sisepuede_item"] = cw_df["sisepuede_item"].apply(lambda x : f"yf_agrc_{x}_tonne_ha")
cw_df.head()

,Item,cat_1,sisepuede_item,super_cat
0,"Abaca, manila hemp, raw",fibers,yf_agrc_fibers_tonne_ha,fibers
1,Agave fibres nes,fibers,yf_agrc_fibers_tonne_ha,fibers
2,"Agave fibres, raw, n.e.c.",fibers,yf_agrc_fibers_tonne_ha,fibers
3,"Almonds, in shell",nuts,yf_agrc_nuts_tonne_ha,woody_perennial
4,"Almonds, with shell",nuts,yf_agrc_nuts_tonne_ha,woody_perennial


In [8]:
# Creating crosswalk dictionary
cw_dict = dict(zip(cw_df['Item'], cw_df['sisepuede_item']))
# cw_dict

In [9]:
# Adding a column with sisepuede_items input style using the cw dictionary
yf_df["sisepuede_item"] = yf_df["Item"].replace(cw_dict)

# Queries all data except where item == Mate
yf_df = yf_df.query("Item != 'Maté'")

yf_df.head()

,iso_code3,Nation,Item,Year,Value,sisepuede_item
0,AFG,Afghanistan,"Almonds, in shell",2011,4500.0,yf_agrc_nuts_tonne_ha
1,AFG,Afghanistan,"Almonds, in shell",2012,4596.0,yf_agrc_nuts_tonne_ha
2,AFG,Afghanistan,"Almonds, in shell",2013,2991.0,yf_agrc_nuts_tonne_ha
3,AFG,Afghanistan,"Almonds, in shell",2014,1999.6,yf_agrc_nuts_tonne_ha
4,AFG,Afghanistan,"Almonds, in shell",2015,1652.1,yf_agrc_nuts_tonne_ha


In [10]:
# Checking that all the cw mapping was done for all items
yf_df.isnull().sum()

iso_code3         0
Nation            0
Item              0
Year              0
Value             0
sisepuede_item    0
dtype: int64

In [11]:
# Performing groupby since many items from FAO where mapped to the same category
# yf_df.groupby(["Nation","Year","sisepuede_item"])["Value"].mean()

yf_df_grouped = yf_df.groupby(["iso_code3", "Nation","Year","sisepuede_item"])["Value"].mean().reset_index() 
yf_df_grouped.head()

,iso_code3,Nation,Year,sisepuede_item,Value
0,AFG,Afghanistan,2011,yf_agrc_cereals_tonne_ha,1941.975000
1,AFG,Afghanistan,2011,yf_agrc_fibers_tonne_ha,1000.000000
2,AFG,Afghanistan,2011,yf_agrc_fruits_tonne_ha,7305.607692
3,AFG,Afghanistan,2011,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,641.400000
4,AFG,Afghanistan,2011,yf_agrc_nuts_tonne_ha,3167.300000


In [12]:
yf_df_pivot = yf_df_grouped.pivot(index=['iso_code3', 'Nation', 'Year'], columns='sisepuede_item', values='Value').reset_index()  
yf_df_pivot.head()

sisepuede_item,iso_code3,Nation,Year,yf_agrc_bevs_and_spices_tonne_ha,yf_agrc_cereals_tonne_ha,yf_agrc_fibers_tonne_ha,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,AFG,Afghanistan,2011,NaN,1941.975,1000.0,7305.607692,641.4,3167.300,1383.650000,NaN,827.2,3200.0,30000.0,11500.00,8689.96
1,AFG,Afghanistan,2012,NaN,2252.250,1100.0,7737.607692,675.7,3195.475,1645.850000,NaN,798.9,3429.3,25000.0,11476.20,8601.68
2,AFG,Afghanistan,2013,NaN,2167.725,1161.8,7541.530769,675.7,2479.775,1645.850000,NaN,809.6,3726.8,28000.0,14131.15,9019.42
3,AFG,Afghanistan,2014,NaN,2129.475,1223.7,7407.823077,716.7,2157.600,1296.700000,NaN,752.4,3643.2,23390.0,12583.30,10405.68
4,AFG,Afghanistan,2015,NaN,2022.725,1198.4,7303.730769,720.0,1341.825,937.233333,NaN,742.9,3731.7,14471.7,10624.10,8312.70


In [13]:
# Create historical input data
sisepuede_items = yf_df_grouped.sisepuede_item.unique()

for sise_var in sisepuede_items:

    # Convertion Kilograms to Ton (metric) : 1 kg = 0.00110231 t
    yf_df_pivot[sise_var] *= 0.00110231 # Change this value if the raw data is in different units

    if yf_df_pivot[sise_var].isna().any():

        yf_df_pivot[sise_var] = yf_df_pivot.groupby(["Nation"])[sise_var].apply(lambda x: x.interpolate().fillna(method='bfill')).reset_index()[sise_var].fillna(0)
    
    dir_path = f'../AFOLU/{sise_var}/input_to_sisepuede'
    yf_df_pivot[["iso_code3","Nation", "Year", sise_var]].to_csv(os.path.join(dir_path,f"historical/{sise_var}.csv"), index = False)

/tmp/ipykernel_72625/3511680694.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  yf_df_pivot[sise_var] = yf_df_pivot.groupby(["Nation"])[sise_var].apply(lambda x: x.interpolate().fillna(method='bfill')).reset_index()[sise_var].fillna(0)
/tmp/ipykernel_72625/3511680694.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  yf_df_pivot[sise_var] = yf_df_pivot.groupby(["Nation"])[sise_var].apply(lambda x: x.interpolate().fillna(method='bfill')).reset_index()[sise_var].fillna(0)
/tmp/ipykernel_72625/3511680694.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  yf_df_pivot[sise_var] = yf_df_pivot.groupby(["Nation"])[sise_var].apply(lambda x: x.interpolate().fillna(method='bfill')).reset_index()[sise_var].fillna(0)
/tmp/ipyke

In [14]:
# Create projected input data
max_year = yf_df_pivot.Year.max()

yf_data_projected = yf_df_pivot.query(f"Year=={max_year}")

yf_data_projected = yf_data_projected.drop(columns = "Year")

df_year = pd.DataFrame({"Year" : range(max_year+1, 2051)})

yf_data_projected = yf_data_projected.merge(df_year, how = "cross") 

for sise_var in sisepuede_items:
    dir_path = f'../AFOLU/{sise_var}/input_to_sisepuede'
    yf_data_projected[["iso_code3","Nation", "Year",sise_var]].to_csv(os.path.join(dir_path,f"projected/{sise_var}.csv"), index = False)

In [18]:
# Checking Iran Data
# yf_df_pivot[yf_df_pivot.iso_code3 == 'IRN'].head(15)

sisepuede_item,iso_code3,Nation,Year,yf_agrc_bevs_and_spices_tonne_ha,yf_agrc_cereals_tonne_ha,yf_agrc_fibers_tonne_ha,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
985,IRN,Iran (Islamic Republic of),2011,0.000000,3.512842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,21.173060,260.077214
986,IRN,Iran (Islamic Republic of),2012,0.000000,3.512842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,13.227720,232.736817
987,IRN,Iran (Islamic Republic of),2013,0.000000,3.512842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,15.248585,239.716313
988,IRN,Iran (Islamic Republic of),2014,0.000000,3.512842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,16.626472,237.056241
989,IRN,Iran (Islamic Republic of),2015,0.000000,3.541281,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,18.243230,249.320740
990,IRN,Iran (Islamic Republic of),2016,0.000000,3.884320,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,16.534650,243.143417
991,IRN,Iran (Islamic Republic of),2017,0.000000,2.866006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,11.059807,222.736440
992,IRN,Iran (Islamic Republic of),2018,0.000000,3.958285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,15.064940,232.339302
993,IRN,Iran (Islamic Republic of),2019,0.000000,5.355022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,20.334864,212.567344
994,IRN,Iran (Islamic Republic of),2020,0.000000,3.440089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,15.567042,214.270700
